In [3]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data

**`Load Data`**
+ `one-hot` format

In [4]:
mnist = input_data.read_data_sets("MNIST_data", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [5]:
print('X_train.shape :', mnist.train.images.shape)
print('Y_train.shape :', mnist.train.labels.shape)

X_train.shape : (55000, 784)
Y_train.shape : (55000, 10)


**`mini batch gradient descent`**

In [6]:
batch_size = 100
n_batch = mnist.train.num_examples // batch_size  # 整除

**`优化程序`**
+ batch_size
+ Neural Network Struct
+ Active Function
+ Cost Function : Cross Entropy
+ 使用其他的优化方式

**`Neural Network Struct`**
+ input  : 784
+ output : 10

In [7]:
x = tf.placeholder(tf.float32, [None, 784])
y = tf.placeholder(tf.float32, [None, 10])

**`Dropout Param`**

In [8]:
keep_prob = tf.placeholder(tf.float32)

**`tf.nn.softmax`**()

In [9]:
Weight_L1 = tf.Variable(tf.truncated_normal([784,300], stddev=0.1))  # (input, output)
Biases_L1 = tf.Variable(tf.zeros([300]) + 0.1)
Z_L1 = tf.matmul(x, Weight_L1) + Biases_L1
A_L1 = tf.nn.tanh(Z_L1)

Drop_L1 = tf.nn.dropout(A_L1, keep_prob)

In [10]:
Weight_L2 = tf.Variable(tf.truncated_normal([300,100], stddev=0.1))  # (input, output)
Biases_L2 = tf.Variable(tf.zeros([100]) + 0.1)
Z_L2 = tf.matmul(Drop_L1, Weight_L2) + Biases_L2
A_L2 = tf.nn.tanh(Z_L2)

Drop_L2 = tf.nn.dropout(A_L2, keep_prob)

In [11]:
Weight_L3 = tf.Variable(tf.truncated_normal([100,10], stddev=0.1))  # (input, output)
Biases_L3 = tf.Variable(tf.zeros([10]) + 0.1)
Z_L3 = tf.matmul(Drop_L2, Weight_L3) + Biases_L3
A_L3 = tf.nn.softmax(Z_L3)

Prediction = tf.nn.softmax(Z_L3)

**`tf.nn.softmax_cross_entropy_with_logits`**()

In [14]:
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y, logits=Prediction))

**`Optimizer`**

In [16]:
lr = tf.Variable(0.001) # 1e-3

# optimizer = tf.train.GradientDescentOptimizer(lr)
optimizer = tf.train.AdamOptimizer(lr)

train = optimizer.minimize(loss)

**`accuary`**
+ **tf.argmax**(input, axis=None) : Returns the index with the largest value across axes of a tensor
+ **tf.equal**(x, y, name=None) : Returns the truth value of (x == y) element-wise.
+ **tf.cast**(x, dtype) : Casts a tensor to a new type. / 类型转换
+ **tf.reduce_mean**(input_tensor, axis=None) : Computes the mean of elements across dimensions of a tensor.

In [17]:
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(Prediction, 1))
    # return a bool-list

accuary = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

**由于该神经网络过于复杂，容易出现overfitting**
+ 因此使用dropout来避免过拟合
+ 当使用dropout之后
    + test accuracy的上升速度变慢，[ 从 96 -> 97 ]， 收敛速度变慢
    + **train accuracy的准确率没有很高**，**test & train accuracy 相对偏差不是很大**

In [18]:
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(30):
        for batch in range(n_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            sess.run(train, feed_dict={
                x : batch_xs,
                y : batch_ys,
                keep_prob : 1.0
            })
        #if epoch % 100 == 0:
            # keep_prob
            # 1.0 : 所有神经元都在使用
            
        test_acc = sess.run(accuary, feed_dict={
            x:mnist.test.images,
            y:mnist.test.labels,
            keep_prob : 1.0
        })
        train_acc = sess.run(accuary, feed_dict={
            x:mnist.train.images,
            y:mnist.train.labels,
            keep_prob : 1.0
        })

        print("Iter " + str(epoch) + "  Test Acuuary " + str(test_acc) + "  Train Acuuary " + str(train_acc))

Iter 0  Test Acuuary 0.944  Train Acuuary 0.9482
Iter 1  Test Acuuary 0.9591  Train Acuuary 0.9644
Iter 2  Test Acuuary 0.9663  Train Acuuary 0.9726909
Iter 3  Test Acuuary 0.9676  Train Acuuary 0.9778727
Iter 4  Test Acuuary 0.9714  Train Acuuary 0.98210907
Iter 5  Test Acuuary 0.9696  Train Acuuary 0.9814909
Iter 6  Test Acuuary 0.9748  Train Acuuary 0.98725456
Iter 7  Test Acuuary 0.9753  Train Acuuary 0.9892
Iter 8  Test Acuuary 0.975  Train Acuuary 0.9886364
Iter 9  Test Acuuary 0.977  Train Acuuary 0.99047273
Iter 10  Test Acuuary 0.9755  Train Acuuary 0.99125457
Iter 11  Test Acuuary 0.9753  Train Acuuary 0.9913273
Iter 12  Test Acuuary 0.9757  Train Acuuary 0.99234545
Iter 13  Test Acuuary 0.9785  Train Acuuary 0.99363637
Iter 14  Test Acuuary 0.9779  Train Acuuary 0.99314547
Iter 15  Test Acuuary 0.9762  Train Acuuary 0.9930182
Iter 16  Test Acuuary 0.9779  Train Acuuary 0.99398184
Iter 17  Test Acuuary 0.9777  Train Acuuary 0.99472725
Iter 18  Test Acuuary 0.9764  Train Acuua

In [19]:
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(30):
        for batch in range(n_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            sess.run(train, feed_dict={
                x : batch_xs,
                y : batch_ys,
                keep_prob : 0.7
            })
        #if epoch % 100 == 0:
            # keep_prob
            # 1.0 : 所有神经元都在使用
            
        test_acc = sess.run(accuary, feed_dict={
            x:mnist.test.images,
            y:mnist.test.labels,
            keep_prob : 1.0
        })
        train_acc = sess.run(accuary, feed_dict={
            x:mnist.train.images,
            y:mnist.train.labels,
            keep_prob : 1.0
        })

        print("Iter " + str(epoch) + "  Test Acuuary " + str(test_acc) + "  Train Acuuary " + str(train_acc))

Iter 0  Test Acuuary 0.9334  Train Acuuary 0.93316364
Iter 1  Test Acuuary 0.944  Train Acuuary 0.94725454
Iter 2  Test Acuuary 0.9547  Train Acuuary 0.9575091
Iter 3  Test Acuuary 0.9563  Train Acuuary 0.9615818
Iter 4  Test Acuuary 0.9607  Train Acuuary 0.9653091
Iter 5  Test Acuuary 0.9605  Train Acuuary 0.96694547
Iter 6  Test Acuuary 0.9633  Train Acuuary 0.9695455
Iter 7  Test Acuuary 0.9671  Train Acuuary 0.9734727
Iter 8  Test Acuuary 0.9642  Train Acuuary 0.97174543
Iter 9  Test Acuuary 0.9687  Train Acuuary 0.97676367
Iter 10  Test Acuuary 0.9704  Train Acuuary 0.9770182
Iter 11  Test Acuuary 0.9696  Train Acuuary 0.9778
Iter 12  Test Acuuary 0.9697  Train Acuuary 0.97801816
Iter 13  Test Acuuary 0.9717  Train Acuuary 0.9802909
Iter 14  Test Acuuary 0.9705  Train Acuuary 0.9795455
Iter 15  Test Acuuary 0.9737  Train Acuuary 0.9821454
Iter 16  Test Acuuary 0.9711  Train Acuuary 0.982
Iter 17  Test Acuuary 0.9716  Train Acuuary 0.98145455
Iter 18  Test Acuuary 0.9726  Train Acu